# Day 8: Handheld Halting

https://adventofcode.com/2020/day/8

- `acc` increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.

- `jmp` jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.

- `nop` stands for No OPeration - it does nothing. The instruction immediately below it is executed next.

## Part 1

In [2]:
def readInstructions(filename):
    with open(filename) as f:
        instr = [ [v[0],int(v[1])] for v in [ l.strip('\n').split(' ') for l in f.readlines()] ]
    return instr

In [3]:
def runInstructions(instr):
    
    accumul = 0
    ncmd = len(instr)
    ccmd = [0] * ncmd # count how many times an instruction has been executed  
    icmd = 0
    
    while(True):

        [command,value] = instr[icmd]
        ccmd[icmd] += 1
        # print(icmd, command, value, ccmd[icmd], accumul)
        
        if ccmd[icmd] == 2:
            #print("Reached second execution of instruction",icmd)
            #print("Accumulator =", accumul)
            return False, accumul
        
        if command=="acc":
            accumul += value
            icmd += 1
        if command=='jmp':
            icmd += value
        if command=="nop":
            # do nothing
            icmd += 1

        if icmd == ncmd:
            #print("Accumulator = ", accumul)
            #print("Reach end of instructions. Program ended")
            return True, accumul

In [4]:
instr_test = readInstructions("data/day08test0.txt")
status, accumul = runInstructions(instr_test)
print("Accumulator =",accumul)

Accumulator = 5


In [5]:
instr = readInstructions("data/input08.txt")
status, accumul = runInstructions(instr)
print("Accumulator =",accumul)

Accumulator = 1753


In [6]:
instr_new = list(instr_test) # make a copy of the instructions
instr_new[7][0] = 'nop'
status, accumul = runInstructions(instr_test)
print("Accumulator =",accumul)

Accumulator = 8


## Brute forcing Part 2...

In [7]:
#instr_orig = readInstructions("data/day08test0.txt")
instr_orig = readInstructions("data/input08.txt")

# be careful with shallow vs deep copy of the instructions, since they are an list of lists 
# (i.e. an iterable of iterables): a simple copy() would make two different lists of the external list, but 
# internal lists [command, value] would be the same objects

import copy

for i in range(len(instr_orig)):

    instr_new = copy.deepcopy(instr_orig)
        
    if instr_orig[i][0] == 'jmp':
        instr_new[i] = [ 'nop', instr_orig[i][1] ] 
    if instr_orig[i][0] == 'nop':
        instr_new[i] = [ 'jmp', instr_orig[i][1] ] 
    
    #print(i, instr_orig[i],instr_new[i])

    status, accumul = runInstructions(instr_new)
    
    if status:
        print("Accumulator =",accumul)

Accumulator = 733


## Refactoring interpreter in a class

This looks like a programming language we might need to extend in the coming days, a bit like the Intcode computer of 2019, so I better begin to refactorize it in a class from the beginning...

In [8]:
import numpy as np

class interpreter():
    """Code Interpreter"""

    def __init__(self, instr, phase=0, name="AMP", debug=False):

        self.instr = np.array(instr)
        self.debug = debug
        self.icmd = 0
        self.accumul = 0
        self.ncmd = len(instr)
        self.ccmd = [0] * self.ncmd # count how many times an instruction has been executed  
    
    def getAcc(self):
        return self.accumul

    def run(self):
    
        while(True):
            
            [command,value] = self.instr[self.icmd]
            value = int(value)
            
            #if self.debug:
            #    print(self.icmd,command,value)
            
            self.ccmd[self.icmd] += 1
        
            if self.ccmd[self.icmd] == 2:
                if self.debug:
                    print("Reached second execution of instruction",self.icmd)
                return False
        
            if command=="acc":
                self.accumul += value
                self.icmd += 1
            if command=='jmp':
                self.icmd += value
            if command=="nop":
                # do nothing
                self.icmd += 1

            if self.icmd == self.ncmd:
                if self.debug:
                    print("Reach end of instructions. Program ended.")
                return True

In [10]:
instr_test = readInstructions("data/day08test0.txt")

c1 = interpreter(instr_test,debug=True)
c1.run()
print('Accumulator Part 1 Test =',c1.getAcc())

instr_full = readInstructions("data/input08.txt")

c2 = interpreter(instr_full,debug=True)
c2.run()
print('Accumulator Part 1 Full =',c2.getAcc())

Reached second execution of instruction 1
Accumulator Part 1 Test = 5
Reached second execution of instruction 238
Accumulator Part 1 Full = 1753


In [10]:
import copy

instr_test_new = copy.deepcopy(instr_test)
instr_test_new[7][0] = 'nop'

c3 = interpreter(instr_test_new,debug=True)
c3.run()
print('Accumulator Part 2 Test =',c3.getAcc())

Reach end of instructions. Program ended.
Accumulator Part 2 Test = 8


In [34]:
#instr_orig = copy.deepcopy(instr_test)
instr_orig = copy.deepcopy(instr_full)

for i in range(len(instr_orig)):

    instr_new = copy.deepcopy(instr_orig)
     
    if instr_orig[i][0] == 'jmp':
        instr_new[i] = [ 'nop', instr_orig[i][1] ] 
    if instr_orig[i][0] == 'nop':
        instr_new[i] = [ 'jmp', instr_orig[i][1] ] 
    
    c = interpreter(instr_new)
    if c.run():
        print("Accumulator =",c.getAcc())

Accumulator = 733
